# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [3]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [4]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,cuencame de ceniceros,24.8711,-103.6964,34.25,15,75,5.98,MX,1718055375
1,1,west island,-12.1568,96.8225,25.99,89,100,9.77,CC,1718055375
2,2,iskateley,67.6803,53.1512,3.34,88,6,3.58,RU,1718055376
3,3,harstad,68.7990,16.5293,13.66,86,98,2.81,NO,1718055376
4,4,avarua,-21.2078,-159.7750,24.03,53,20,3.09,CK,1718055376


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [5]:
%%capture --no-display

# Configure the map plot
map_plot_city=city_data_df.hvplot.points("Lng","Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    size = "Humidity",
    color = "City"  
)
# YOUR CODE HERE

# Display the map
map_plot_city
# YOUR CODE HERE

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [6]:
# Narrow down cities that fit criteria and drop any results with null values
results_city=city_data_df[(city_data_df['Humidity'] < 70) & 
                                  (city_data_df['Cloudiness'] == 0) & 
                                  (city_data_df['Max Temp'] <= 25) & 
                                  (city_data_df['Max Temp'] >=15)]
# YOUR CODE HERE

# Drop any rows with null values
results_city_null_values= results_city.dropna()
# YOUR CODE HERE

# Display sample data
results_city_null_values
# YOUR CODE HERE

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
39,39,karratha,-20.7377,116.8463,18.78,55,0,2.35,AU,1718055383
41,41,east london,-33.0153,27.9116,16.59,51,0,3.41,ZA,1718055384
62,62,port alfred,-33.5906,26.8910,17.91,52,0,4.78,ZA,1718055384
69,69,burdur,37.7203,30.2908,19.39,40,0,1.54,TR,1718055390
145,145,shkoder,42.0683,19.5126,24.10,52,0,1.62,AL,1718055405
181,181,ayancik,41.9447,34.5861,21.27,67,0,2.51,TR,1718055412
188,188,port saint john's,-31.6229,29.5448,16.40,58,0,2.95,ZA,1718055414
207,207,kapuskasing,49.4169,-82.4331,17.05,39,0,6.17,CA,1718055417
239,239,kutahya,39.4242,29.9833,19.41,47,0,0.32,TR,1718055424
247,247,port shepstone,-30.7414,30.4550,15.52,65,0,3.66,ZA,1718055426


### Step 3: Create a new DataFrame called `hotel_df`.

In [7]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df=results_city[['City', 'Lat', 'Lng', 'Humidity']].copy()
# YOUR CODE HERE 'Country'

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name']=''
# YOUR CODE HERE

# Display sample data
hotel_df
# YOUR CODE HERE

,City,Lat,Lng,Humidity,Hotel Name
39,karratha,-20.7377,116.8463,55,
41,east london,-33.0153,27.9116,51,
62,port alfred,-33.5906,26.8910,52,
69,burdur,37.7203,30.2908,40,
145,shkoder,42.0683,19.5126,52,
181,ayancik,41.9447,34.5861,67,
188,port saint john's,-31.6229,29.5448,58,
207,kapuskasing,49.4169,-82.4331,39,
239,kutahya,39.4242,29.9833,47,
247,port shepstone,-30.7414,30.4550,65,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [8]:
# Set parameters to search for a hotel
radius = 1000# YOUR CODE HERE
params = {"apiKey": geoapify_key,
    "limit":20,
    'categories':'accommodation.hotel'
    # YOUR CODE HERE
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row["Lat"]
    longitude = row["Lng"]
    # YOUR CODE HERE
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"# YOUR CODE HERE
    params["bias"] = f"proximity:{longitude},{latitude}"# YOUR CODE HERE
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)# YOUR CODE HERE
    
    # Convert the API response to JSON format
    name_address = name_address.json()# YOUR CODE HERE
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
karratha - nearest hotel: Karratha International Hotel
east london - nearest hotel: No hotel found
port alfred - nearest hotel: No hotel found
burdur - nearest hotel: Özeren
shkoder - nearest hotel: Hotel Rozafa
ayancik - nearest hotel: No hotel found
port saint john's - nearest hotel: Outback Inn
kapuskasing - nearest hotel: Travelodge
kutahya - nearest hotel: Hilton Garden Inn Kutahya
port shepstone - nearest hotel: No hotel found
al bardiyah - nearest hotel: فندق البردي
puke - nearest hotel: Hotel Turizmi Pukë
holland - nearest hotel: Haworth Hotel
tindouf - nearest hotel: محمد بوسبي
sinop - nearest hotel: No hotel found
box elder - nearest hotel: No hotel found
tobruk - nearest hotel: فندق زهرة البطنان -طبرق


,City,Lat,Lng,Humidity,Hotel Name
39,karratha,-20.7377,116.8463,55,Karratha International Hotel
41,east london,-33.0153,27.9116,51,No hotel found
62,port alfred,-33.5906,26.8910,52,No hotel found
69,burdur,37.7203,30.2908,40,Özeren
145,shkoder,42.0683,19.5126,52,Hotel Rozafa
181,ayancik,41.9447,34.5861,67,No hotel found
188,port saint john's,-31.6229,29.5448,58,Outback Inn
207,kapuskasing,49.4169,-82.4331,39,Travelodge
239,kutahya,39.4242,29.9833,47,Hilton Garden Inn Kutahya
247,port shepstone,-30.7414,30.4550,65,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [9]:
%%capture --no-display

# Configure the map plot
map_plot_city2 = hotel_df.hvplot.points("Lng","Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height =600,
    size = "Humidity",
    color = "City",
    hover_cols = ['City', 'Country', 'Hotel Name'] 
)
# YOUR CODE HERE

# Display the map
map_plot_city2
# YOUR CODE HERE

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name)